In [ ]:
import pandas as pd
import json
from pathlib import Path

In [ ]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

print("드라이브 마운트 완료!")

Mounted at /content/drive
드라이브 마운트 완료!


In [ ]:
# 데이터셋 경로
dataset_path = '/content/drive/MyDrive/Colab Notebooks/woke-odds/pairwise_comparison_results.csv'

# CSV 파일 로드
df = pd.read_csv(dataset_path)

print(f"전체 데이터 개수: {len(df)}")
print(f"\n컬럼 목록: {df.columns.tolist()}")
print(f"\n데이터셋 샘플:")
df.head()

전체 데이터 개수: 1601

컬럼 목록: ['query', 'original_question', 'category', 'subclass', 'question_1', 'question_2', 'winner']

데이터셋 샘플:


,query,original_question,category,subclass,question_1,question_2,winner
0,[AO|WHOM] Give me a list of good coffee shops?,Give me a list of good coffee shops?,AO,WHOM,Question: What is the city or country of your ...,WHOM is a location?,Question 1
1,[AO|WHOM] Give me some Mother's Day gift ideas.,Give me some Mother's Day gift ideas.,AO,WHOM,When do you come in with me?,"What type of gift do you mean, such as persona...",Question 2
2,[AO|WHAT] Help me come up with 3 ideas for a n...,Help me come up with 3 ideas for a new busines...,AO,WHAT,WHAT is one of the businesses you want to start?,Which part would you like assistance with: dev...,Question 2
3,[AO|WHOM] Write the first paragraph of a blog ...,Write the first paragraph of a blog post descr...,AO,WHOM,WHAT was the name of his first dog?,END,Question 2
4,[AO|WHOM] Give me some tips on how to train fo...,Give me some tips on how to train for a marathon.,AO,WHOM,I'm not sure if you're asking for the training...,"For example, ""WHOM_???"" to specify whether it'...",Question 2


In [ ]:
# 데이터셋 경로
dataset_path = '/content/drive/MyDrive/Colab Notebooks/woke-odds/pairwise_comparison_results.csv'

# CSV 파일 로드
df = pd.read_csv(dataset_path)

print(f"전체 데이터 개수: {len(df)}")
print(f"\n컬럼 목록: {df.columns.tolist()}")

# 카테고리 종류 확인
print(f"\n=== 카테고리 종류 ===")
print(df['category'].unique())
print(f"\n카테고리별 개수:")
print(df['category'].value_counts())

print(f"\n=== 서브클래스 종류 ===")
print(df['subclass'].unique())
print(f"\n서브클래스별 개수:")
print(df['subclass'].value_counts())

print(f"\n데이터셋 샘플:")
df.head()

전체 데이터 개수: 1601

컬럼 목록: ['query', 'original_question', 'category', 'subclass', 'question_1', 'question_2', 'winner']

=== 카테고리 종류 ===
['AO' 'EM' 'LA']

카테고리별 개수:
category
AO    801
EM    400
LA    400
Name: count, dtype: int64

=== 서브클래스 종류 ===
['WHOM' 'WHAT' 'WHEN' 'WHERE' 'UNF' 'CONT' 'SEM' 'LEX']

서브클래스별 개수:
subclass
WHAT     201
WHOM     200
WHEN     200
WHERE    200
UNF      200
CONT     200
SEM      200
LEX      200
Name: count, dtype: int64

데이터셋 샘플:


,query,original_question,category,subclass,question_1,question_2,winner
0,[AO|WHOM] Give me a list of good coffee shops?,Give me a list of good coffee shops?,AO,WHOM,Question: What is the city or country of your ...,WHOM is a location?,Question 1
1,[AO|WHOM] Give me some Mother's Day gift ideas.,Give me some Mother's Day gift ideas.,AO,WHOM,When do you come in with me?,"What type of gift do you mean, such as persona...",Question 2
2,[AO|WHAT] Help me come up with 3 ideas for a n...,Help me come up with 3 ideas for a new busines...,AO,WHAT,WHAT is one of the businesses you want to start?,Which part would you like assistance with: dev...,Question 2
3,[AO|WHOM] Write the first paragraph of a blog ...,Write the first paragraph of a blog post descr...,AO,WHOM,WHAT was the name of his first dog?,END,Question 2
4,[AO|WHOM] Give me some tips on how to train fo...,Give me some tips on how to train for a marathon.,AO,WHOM,I'm not sure if you're asking for the training...,"For example, ""WHOM_???"" to specify whether it'...",Question 2


In [ ]:
SYSTEM_PROMPT = """You are an AI that generates a single, concise clarifying question when a user's query is ambiguous.

Task:
Generate exactly one clarifying question based on the ambiguity type.

Output format: One clarifying question

Categories:
- EM (Epistemic Misalignment): Questions with unfamiliar entities or self-contradictions
- LA (Linguistic Ambiguity): Questions with lexical or semantic ambiguity
- AO (Aleatoric Output): Questions with missing contextual information causing confusion

Subclasses:
For EM:
- UNF (UNFAMILIAR): Query contains unfamiliar entities or facts
- CONT (CONTRADICTION): Query contains self-contradictions

For LA:
- LEX (LEXICAL): Query contains terms with multiple meanings
- SEM (SEMANTIC): Query lacks context leading to multiple interpretations

For AO:
- WHOM: Query output contains confusion due to missing personal elements
- WHEN: Query output contains confusion due to missing temporal elements
- WHERE: Query output contains confusion due to missing spatial elements
- WHAT: Query output contains confusion due to missing task-specific elements"""

In [ ]:
# question_1, question_2에 숫자 타입이 있는지 확인
print("=== question_1 데이터 타입 확인 ===")
print(df['question_1'].apply(type).value_counts())

print("\n=== question_2 데이터 타입 확인 ===")
print(df['question_2'].apply(type).value_counts())

# 숫자 타입인 row 찾기
print("\n=== 숫자 타입인 question_1 찾기 ===")
numeric_q1 = df[df['question_1'].apply(lambda x: isinstance(x, (int, float)) and not isinstance(x, bool))]
if len(numeric_q1) > 0:
    print(numeric_q1[['query', 'question_1', 'question_2', 'winner']])
else:
    print("없음")

print("\n=== 숫자 타입인 question_2 찾기 ===")
numeric_q2 = df[df['question_2'].apply(lambda x: isinstance(x, (int, float)) and not isinstance(x, bool))]
if len(numeric_q2) > 0:
    print(numeric_q2[['query', 'question_1', 'question_2', 'winner']])
else:
    print("없음")

# NaN 값 확인
print("\n=== NaN 값 확인 ===")
print(f"question_1 NaN 개수: {df['question_1'].isna().sum()}")
print(f"question_2 NaN 개수: {df['question_2'].isna().sum()}")

=== question_1 데이터 타입 확인 ===
question_1
<class 'str'>      1410
<class 'float'>     191
Name: count, dtype: int64

=== question_2 데이터 타입 확인 ===
question_2
<class 'str'>      1399
<class 'float'>     202
Name: count, dtype: int64

=== 숫자 타입인 question_1 찾기 ===
                                                  query question_1  \
12    [AO|WHOM] Give me a list of the 3 best activit...        NaN   
42                  [AO|WHOM] list 7 favourite flavours        NaN   
67    [AO|WHOM] Give me a list of the most agreed up...        NaN   
88              [AO|WHOM] Name some famous anime shows?        NaN   
91               [AO|WHOM] Name some famous jazz songs?        NaN   
...                                                 ...        ...   
1580        [LA|LEX] What are some foods that are rich?        NaN   
1584       [LA|LEX] What does it mean to "try" someone?        NaN   
1589   [LA|LEX] What are some common uses for a pocket?        NaN   
1593  [LA|LEX] What is the most common us

In [ ]:
print("=== question_1 데이터 타입 확인 ===")
print(df['question_1'].apply(type).value_counts())

print("\n=== question_2 데이터 타입 확인 ===")
print(df['question_2'].apply(type).value_counts())

=== question_1 데이터 타입 확인 ===
question_1
<class 'str'>      1410
<class 'float'>     191
Name: count, dtype: int64

=== question_2 데이터 타입 확인 ===
question_2
<class 'str'>      1399
<class 'float'>     202
Name: count, dtype: int64


In [ ]:
# float 타입인 값들이 실제로 NaN인지 확인
print("=== float 타입인 question_1 확인 ===")
float_q1 = df[df['question_1'].apply(lambda x: isinstance(x, float))]
print(f"총 {len(float_q1)}개")
print(float_q1[['query', 'question_1', 'question_2', 'winner']].head(10))

print("\n=== float 타입인 question_2 확인 ===")
float_q2 = df[df['question_2'].apply(lambda x: isinstance(x, float))]
print(f"총 {len(float_q2)}개")
print(float_q2[['query', 'question_1', 'question_2', 'winner']].head(10))

# 실제 NaN 개수
print("\n=== NaN 개수 ===")
print(f"question_1 NaN: {df['question_1'].isna().sum()}개")
print(f"question_2 NaN: {df['question_2'].isna().sum()}개")

=== float 타입인 question_1 확인 ===
총 191개
                                                 query question_1  \
12   [AO|WHOM] Give me a list of the 3 best activit...        NaN   
42                 [AO|WHOM] list 7 favourite flavours        NaN   
67   [AO|WHOM] Give me a list of the most agreed up...        NaN   
88             [AO|WHOM] Name some famous anime shows?        NaN   
91              [AO|WHOM] Name some famous jazz songs?        NaN   
139  [AO|WHOM] Give me a list of things I should br...        NaN   
140  [AO|WHERE] How can I drive from South Bay to S...        NaN   
145  [AO|WHOM] Give me a list of six different type...        NaN   
147  [AO|WHAT] Tell me three key things to do to dr...        NaN   
152  [AO|WHEN] Name some NHL hockey players that we...        NaN   

                                            question_2      winner  
12                                       none rephrase  Question 2  
42                                                 NaN  Questio

In [ ]:
def create_dpo_entry(row):
    """
    단일 행을 DPO 형식으로 변환 (수정된 필터링)

    Args:
        row: DataFrame의 한 행

    Returns:
        dict: DPO 형식의 데이터 (prompt, chosen, rejected)
    """
    # query 필드 사용 (모델에게 전달되는 입력)
    user_query = row['query']

    # winner에 따라 chosen/rejected 결정
    winner = row['winner']
    question_1 = row['question_1']
    question_2 = row['question_2']

    # === 필터링 1: NaN 체크 - 둘 다 NaN이면 스킵 ===
    q1_is_nan = pd.isna(question_1)
    q2_is_nan = pd.isna(question_2)

    if q1_is_nan and q2_is_nan:
        return None

    # 문자열 변환 및 공백 제거
    question_1 = str(question_1).strip() if not q1_is_nan else ""
    question_2 = str(question_2).strip() if not q2_is_nan else ""

    # === 필터링 2: 이상한 시작 패턴 - 둘 다 해당되면 스킵 ===
    invalid_starts = [
        'NONE', 'none', 'None',
        'EMPTY', 'empty', 'Empty',
        'END', 'end',
        'nan',
    ]

    q1_bad_start = any(question_1.startswith(pattern) for pattern in invalid_starts) if question_1 else False
    q2_bad_start = any(question_2.startswith(pattern) for pattern in invalid_starts) if question_2 else False

    if q1_bad_start and q2_bad_start:
        return None

    # === 필터링 3: 이상한 포함 패턴 - 둘 다 해당되면 스킵 ===
    invalid_contains = [
        '###',
        '<think>',
        '>@',
        '|minort|',
        '<|output|>',
        'baaaaaaa',
        '多少',
        '请再',
    ]

    q1_bad_content = any(pattern in question_1 for pattern in invalid_contains) if question_1 else False
    q2_bad_content = any(pattern in question_2 for pattern in invalid_contains) if question_2 else False

    if q1_bad_content and q2_bad_content:
        return None

    # winner 값에 따라 chosen/rejected 할당
    if winner == 'Question 1':
        chosen = question_1
        rejected = question_2
    elif winner == 'Question 2':
        chosen = question_2
        rejected = question_1
    else:
        return None

    # chosen이 비어있으면 스킵
    if not chosen or chosen == 'nan':
        return None

    # DPO 형식으로 구성
    dpo_entry = {
        "prompt": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_query}
        ],
        "chosen": [
            {"role": "assistant", "content": chosen}
        ],
        "rejected": [
            {"role": "assistant", "content": rejected}
        ]
    }

    return dpo_entry

In [ ]:
# 데이터 변환
dpo_data = []
skipped_count = 0

for idx, row in df.iterrows():
    dpo_entry = create_dpo_entry(row)

    if dpo_entry is not None:
        dpo_data.append(dpo_entry)
    else:
        skipped_count += 1

print(f"변환 완료!")
print(f"총 변환된 데이터: {len(dpo_data)}개")
print(f"스킵된 데이터: {skipped_count}개")

변환 완료!
총 변환된 데이터: 1405개
스킵된 데이터: 196개


In [ ]:
# 첫 번째 샘플 출력
if dpo_data:
    print("="*80)
    print("첫 번째 샘플")
    print("="*80)
    sample = dpo_data[0]

    print("\n[Prompt]")
    for msg in sample['prompt']:
        print(f"  {msg['role']}: {msg['content'][:100]}...")

    print("\n[Chosen]")
    print(f"  {sample['chosen'][0]['content']}")

    print("\n[Rejected]")
    print(f"  {sample['rejected'][0]['content']}")

    print("\n" + "="*80)
    print("두 번째 샘플")
    print("="*80)
    sample = dpo_data[1]

    print("\n[Prompt - User Query]")
    print(f"  {sample['prompt'][1]['content']}")

    print("\n[Chosen]")
    print(f"  {sample['chosen'][0]['content']}")

    print("\n[Rejected]")
    print(f"  {sample['rejected'][0]['content']}")

첫 번째 샘플

[Prompt]
  system: You are an AI that generates a single, concise clarifying question when a user's query is ambiguous....
  user: [AO|WHOM] Give me a list of good coffee shops?...

[Chosen]
  Question: What is the city or country of your favorite coffee shop?

[Rejected]
  WHOM is a location?

두 번째 샘플

[Prompt - User Query]
  [AO|WHOM] Give me some Mother's Day gift ideas.

[Chosen]
  What type of gift do you mean, such as personal, material, or experiences?

[Rejected]
  When do you come in with me?


In [ ]:
# DPO 데이터에서 chosen과 rejected의 content 타입 확인
print("=" * 80)
print("DPO 데이터 타입 확인")
print("=" * 80)

# chosen content 타입 추출
chosen_types = []
rejected_types = []

for entry in dpo_data:
    chosen_content = entry['chosen'][0]['content']
    rejected_content = entry['rejected'][0]['content']

    chosen_types.append(type(chosen_content))
    rejected_types.append(type(rejected_content))

# 타입별 개수 확인
import pandas as pd
from collections import Counter

print("\n=== Chosen Content 타입 ===")
chosen_counter = Counter(chosen_types)
for dtype, count in chosen_counter.items():
    print(f"{dtype}: {count}개")

print("\n=== Rejected Content 타입 ===")
rejected_counter = Counter(rejected_types)
for dtype, count in rejected_counter.items():
    print(f"{dtype}: {count}개")

# float 타입 샘플 확인
print("\n" + "=" * 80)
print("Float 타입 샘플 확인")
print("=" * 80)

float_samples = []
for idx, entry in enumerate(dpo_data):
    chosen_content = entry['chosen'][0]['content']
    rejected_content = entry['rejected'][0]['content']

    if isinstance(chosen_content, float) or isinstance(rejected_content, float):
        float_samples.append({
            'index': idx,
            'query': entry['prompt'][1]['content'][:80],
            'chosen': chosen_content,
            'chosen_type': type(chosen_content),
            'rejected': rejected_content,
            'rejected_type': type(rejected_content)
        })

        if len(float_samples) >= 5:  # 처음 5개만
            break

if float_samples:
    print(f"\nFloat 타입 발견: {len(float_samples)}개 샘플\n")
    for sample in float_samples:
        print(f"Index: {sample['index']}")
        print(f"Query: {sample['query']}...")
        print(f"Chosen: {sample['chosen']} (type: {sample['chosen_type']})")
        print(f"Rejected: {sample['rejected']} (type: {sample['rejected_type']})")
        print("-" * 40)
else:
    print("\n✅ Float 타입 없음! 모두 str 타입입니다.")

DPO 데이터 타입 확인

=== Chosen Content 타입 ===
<class 'str'>: 1405개

=== Rejected Content 타입 ===
<class 'str'>: 1405개

Float 타입 샘플 확인

✅ Float 타입 없음! 모두 str 타입입니다.


In [ ]:
# 저장 경로 설정
output_dir = '/content/drive/MyDrive/Colab Notebooks/woke-odds/'
output_filename = 'dpo_dataset_from_pairwise_edited.jsonl'
output_path = output_dir + output_filename

# JSONL 형식으로 저장
with open(output_path, 'w', encoding='utf-8') as f:
    for entry in dpo_data:
        json_line = json.dumps(entry, ensure_ascii=False)
        f.write(json_line + '\n')

print(f"저장 완료!")
print(f"파일 경로: {output_path}")
print(f"총 {len(dpo_data)}개의 데이터가 저장되었습니다.")

저장 완료!
파일 경로: /content/drive/MyDrive/Colab Notebooks/woke-odds/dpo_dataset_from_pairwise_edited.jsonl
총 1405개의 데이터가 저장되었습니다.


In [ ]:
# 저장된 파일 첫 3줄 확인
print("="*80)
print("저장된 JSONL 파일 샘플 (첫 3줄)")
print("="*80)

with open(output_path, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i >= 3:
            break
        data = json.loads(line)
        print(f"\n--- Line {i+1} ---")
        print(f"User Query: {data['prompt'][1]['content']}")
        print(f"Chosen: {data['chosen'][0]['content']}")
        print(f"Rejected: {data['rejected'][0]['content']}")

저장된 JSONL 파일 샘플 (첫 3줄)

--- Line 1 ---
User Query: [AO|WHOM] Give me a list of good coffee shops?
Chosen: Question: What is the city or country of your favorite coffee shop?
Rejected: WHOM is a location?

--- Line 2 ---
User Query: [AO|WHOM] Give me some Mother's Day gift ideas.
Chosen: What type of gift do you mean, such as personal, material, or experiences?
Rejected: When do you come in with me?

--- Line 3 ---
User Query: [AO|WHAT] Help me come up with 3 ideas for a new business and how I can start it
Chosen: Which part would you like assistance with: developing new business ideas or outlining the steps to start a business?
Rejected: WHAT is one of the businesses you want to start?
